In [1]:
from cmethods.cmethods2 import CMethods2 as cm2

In [3]:
import xarray as xr

In [4]:
obs = xr.open_dataset("examples/input_data/observations.nc")
simh = xr.open_dataset("examples/input_data/control.nc")
simp = xr.open_dataset("examples/input_data/scenario.nc")

In [6]:
cm2.adjust(method="quantile_delta_mapping",obs=obs,simh=simh,simp=simp, kind="+", n_quantiles=100)

<xarray.Dataset>
Dimensions:  (lat: 4, lon: 2, time: 10950)
Coordinates:
  * lat      (lat) int64 23 24 25 26
  * lon      (lon) int64 0 1
  * time     (time) object 2001-01-01 00:00:00 ... 2030-12-31 00:00:00
Data variables:
    tas      (time, lat, lon) float64 -23.1 -22.1 -24.44 ... -29.99 -28.99

In [8]:
cm2.adjust(method="delta_method",obs=obs,simh=simh,simp=simp,group="time.month",kind="+")

<xarray.Dataset>
Dimensions:  (time: 10950, lat: 4, lon: 2)
Coordinates:
  * lat      (lat) int64 23 24 25 26
  * lon      (lon) int64 0 1
  * time     (time) object 2001-01-01 00:00:00 ... 2030-12-31 00:00:00
Data variables:
    tas      (time, lat, lon) float64 -23.09 -22.09 -24.45 ... -29.83 -28.83

In [6]:
def apply_ufunc(method, obs, simh, simp, **kwargs):
    result = xr.apply_ufunc(
        cm.delta_method,
        obs,
        simh,
        # Need to spoof a fake time axis since 'time' coord on full dataset is different
        # than 'time' coord on training dataset.
        simp.rename({"time": "t2"}),
        dask="parallelized",
        vectorize=True,
        # This will vectorize over the time dimension, so will submit each grid cell
        # independently
        input_core_dims=[["time"], ["time"], ["t2"]],
        # Need to denote that the final output dataset will be labeled with the
        # spoofed time coordinate
        output_core_dims=[["t2"]],
        kwargs=dict(kwargs),
    )

    # Rename to proper coordinate name.
    result = result.rename({"t2": "time"})

    # ufunc will put the core dimension to the end (time), so want to preserve original
    # order where time is commonly first.
    return result.transpose(*obs.dims)

In [12]:
group = "time.month"
method = "delta_method"



In [15]:
def grouped_adjustment(cls, method, obs, simh, simp, **kwargs):
    obs_g = list(obs.groupby("time.month"))
    simh_g = list(simh.groupby("time.month"))
    simp_g = list(simp.groupby("time.month"))
    
    result = None
    for month in range(len(list(obs_g))):
        obs_gds = obs_g[month][1]
        simh_gds = simh_g[month][1]
        simp_gds = simp_g[month][1]
        
        monthly_result = cls.apply_ufunc(method, obs_gds, simh_gds, simp_gds, kwargs)
        if result is None:
            result = monthly_result
        else:
            result = xr.concat([result, monthly_result], dim="time")
    return result

In [14]:
result

<xarray.Dataset>
Dimensions:  (time: 10950, lat: 4, lon: 2)
Coordinates:
  * lat      (lat) int64 23 24 25 26
  * lon      (lon) int64 0 1
  * time     (time) object 2001-01-01 00:00:00 ... 2030-12-31 00:00:00
Data variables:
    tas      (time, lat, lon) float64 -23.09 -22.09 -24.45 ... -29.83 -28.83